## LSTM and CNN results comparison

1. We will first calculate the error for each time series
2. Then we will calculate the error for each level in the hierarchy
3. Then we will calculate get the forecasts at the grid level
4. Finally we will calculate the error for the grid level

In [1]:
import sys
sys.path.insert(0, '../')

import src.utils as util
import src.calculate_errors as err
import constants as const

%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
import pandas as pd
import numpy as np

In [4]:
def calculate_errors_per_ts(model_path, ts):
    data = pd.read_csv(f'../ts_data/{ts}.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/final_results/{ts}.csv', index_col=[0])
    test_sample = results_df[['power']].values
    forecasts = results_df[['average_fc']].values
    horizon = 14
    
    mean_mase, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
    return mean_mase, error_dist

In [5]:
def get_results_ts(model_path):
    ts_mase = {}
    ts_dist_err = {}
    mase_vals = []

    for ts in const.TS:
        mean_mase_ts, ts_dist = calculate_errors_per_ts(model_path, ts)
        ts_dist_err[ts] = ts_dist
        ts_mase[ts] = mean_mase_ts
        mase_vals.append(mean_mase_ts)
    return ts_mase, ts_dist_err, mase_vals

In [6]:
lstm_mase_ts, lstm_dist_ts, mase_vals_lstm = get_results_ts('lstm_results')

In [7]:
tcn_mase_ts, tcn_dist_ts, mase_vals_cnn = get_results_ts('cnn_results/tcn')

In [10]:
tcn_d3_mase_ts, tcn_d3_dist_ts, mase_vals_cnn_d3 = get_results_ts('cnn_results/tcn/dilation_3')

In [36]:
tcn_d1_mase_ts, tcn_d1_dist_ts, mase_vals_cnn_d1 = get_results_ts('cnn_results/tcn/dilation_1')

plot error in chart

In [37]:
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=const.TS, y=mase_vals_lstm),
    go.Bar(name='Dilated Convolutions (Rate - 1)', x=const.TS, y=mase_vals_cnn_d1),
    go.Bar(name='Dilated Convolutions (Rate - 2)', x=const.TS, y=mase_vals_cnn),
    go.Bar(name='Dilated Convolutions (Rate - 3)', x=const.TS, y=mase_vals_cnn_d3)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='Mean MASE - 36 samples (1 day ahead - 1 hour resolution)')
fig.show()

Error distributions

In [16]:
fig = go.Figure()
for col in lstm_dist_ts.keys():
    box_data = lstm_dist_ts[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'Time series')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "LSTM", showlegend= False)

In [18]:
fig = go.Figure()
for col in tcn_dist_ts.keys():
    box_data = tcn_dist_ts[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'Time series')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Dilated Convolutions (Rate - 2)", showlegend= False)

In [19]:
fig = go.Figure()
for col in tcn_d3_dist_ts.keys():
    box_data = tcn_d3_dist_ts[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'Time series')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Dilated Convolutions (Rate - 3)", showlegend= False)

## Errors at each level

In [6]:
def get_fc_each_level(start, end, mase_dict):
    if end == -1:
        ts_names = const.TS[start: ]
    else:
        ts_names = const.TS[start: end]
    mase_vals= []
    for ts in ts_names: 
        mase_vals.append(mase_dict[ts])
    avg_mase = np.mean(mase_vals)
    return avg_mase

In [7]:
def get_fc_error_level_model(model_mase_dic):
    level_mase = []
    grid = get_fc_each_level(0, 1, model_mase_dic)
    tl_line = get_fc_each_level(1, 3, model_mase_dic)
    sub = get_fc_each_level(3, 7, model_mase_dic)
    pc = get_fc_each_level(7, 13, model_mase_dic)
    site = get_fc_each_level(13, -1, model_mase_dic)
    level_mase.append(grid)
    level_mase.append(tl_line)
    level_mase.append(sub)
    level_mase.append(pc)
    level_mase.append(site)
    return level_mase

In [8]:
level_results_lstm = get_fc_error_level_model(lstm_dist_ts)

NameError: name 'lstm_dist_ts' is not defined

In [23]:
level_results_lstm

[1.4682344960050517,
 1.2524747891070842,
 1.0567580109364902,
 0.812090928778454,
 0.8593696732847954]

In [24]:
level_results_tcn = get_fc_error_level_model(tcn_dist_ts)

In [25]:
x_vals = ['grid', 'tl', 'sub', 'postcode', 'site']
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=x_vals, y=level_results_lstm),
    go.Bar(name='Dilated Convolutions (Rate -2)', x=x_vals, y=level_results_tcn)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='Avergae MASE at each level - 36 samples per timeseries (1 day ahead - 1 hour resolution)')
fig.show()

## Grid level Forecasts

In [4]:
def sum_fc_results(ts_array, model_path):
    dfs = []
    for ts in ts_array:
        ts_fc = pd.read_csv(f'../{model_path}/final_results/{ts}.csv', index_col=[0])[['average_fc']]
        dfs.append(ts_fc)
    concat_df = pd.concat(dfs, axis=1).sum(axis=1)
    return concat_df

In [5]:
def calculate_grid_error(start, end, model_path):
    if end == -1:
        ts_array = const.TS[start: ]
    else:
        ts_array = const.TS[start: end]
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/final_results/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = sum_fc_results(ts_array, model_path).values
    horizon = 14
    
    mean_mase, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
    return mean_mase, error_dist

In [6]:
def mase_grid(model_path):
    level_mase = []
    level_mase_dist = {}
    grid_mase, grid_dist = calculate_grid_error(0, 1, model_path)
    tl_mase, tl_dist = calculate_grid_error(1, 3, model_path)
    sub_mase, sub_dist = calculate_grid_error(3, 7, model_path)
    pc_mase, pc_dist = calculate_grid_error(7, 13, model_path)
    site_mase, site_dist = calculate_grid_error(13, -1, model_path)
    
    level_mase.append(grid_mase)
    level_mase.append(tl_mase)
    level_mase.append(sub_mase)
    level_mase.append(pc_mase)
    level_mase.append(site_mase)
    
    
    level_mase_dist['grid'] = grid_dist
    level_mase_dist['tl'] = tl_dist
    level_mase_dist['sub'] = sub_dist
    level_mase_dist['pc'] = pc_dist
    level_mase_dist['site'] = site_dist
    
    return level_mase, level_mase_dist

In [7]:
grid_error_naive, grid_error_dist_naive = mase_grid('benchmark_results/naive')

In [8]:
grid_error_naive

[1.079391787981906,
 1.0793917879819062,
 1.0793917879819062,
 1.0793917879819062,
 1.079391787981906]

In [9]:
grid_error_arima, grid_error_dist_arima = mase_grid('benchmark_results/arima')

In [10]:
grid_error_arima

[1.0542174171354008,
 1.0232683119331432,
 1.0241570683528618,
 0.9293869144235187,
 1.0084912706608262]

In [11]:
grid_error_tbats, grid_error_dist_tbats = mase_grid('benchmark_results/tbats')

In [12]:
grid_error_tbats

[1.8748277728587743,
 1.749112915806915,
 1.7063914310919486,
 1.5437416506766883,
 1.683218533714505]

In [13]:
grid_error_prophet, grid_error_dist_prophet = mase_grid('benchmark_results/prophet')

In [14]:
grid_error_prophet

[1.9623944934883193,
 1.9665700550555831,
 1.966532036884487,
 1.0803983146720393,
 1.0899355671492141]

In [15]:
grid_error, grid_error_dist = mase_grid('lstm_results')

In [16]:
grid_error

[1.468234482294422,
 1.416183866343975,
 1.330124385681185,
 1.123227003082377,
 1.0130934871185597]

In [29]:
grid_error_lstm2, grid_error_dist_lstm2 = mase_grid('dnn_results/lstm')

In [30]:
grid_error_lstm2

[2.2111054795723497,
 2.1320336417143246,
 2.197662632715515,
 3.4974980491548475,
 3.4665786991154675]

In [17]:
grid_error_tcn, grid_error_dist_tcn = mase_grid('cnn_results/tcn')

In [18]:
grid_error_tcn

[4.291811358893434,
 4.284694639389833,
 4.287845852587354,
 1.2286401937003335,
 1.2526500181919522]

In [19]:
# create a dataframe
grid_results = grid_error
grid_results.extend(grid_error_tcn)
grid_results.extend(grid_error_arima)
grid_results.extend(grid_error_tbats)
grid_results.extend(grid_error_prophet)
grid_results.extend([grid_error_naive[0]])
len(grid_results)

26

In [20]:
column_names = ['LSTM - Grid', 'LSTM - TL aggregated', 'LSTM - SUB aggregated', 'LSTM - PC aggregated', 
                'LSTM - Site aggregated', 'CNN - Grid', 'CNN - TL aggregated', 'CNN - SUB aggregated', 
                'CNN - PC aggregated', 'CNN - Site aggregated', 'AutoARIMA - Grid', 'AutoARIMA - TL aggregated', 
                'AutoARIMA - SUB aggregated', 
                'AutoARIMA - PC aggregated', 'AutoARIMA - Site aggregated', 'TBATS - Grid', 
                'TBATS - TL aggregated', 'TBATS - SUB aggregated', 
                'TBATS - PC aggregated', 'TBATS - Site aggregated','Prophet - Grid', 
                'Prophet - TL aggregated', 'Prophet - SUB aggregated', 
                'Prophet - PC aggregated', 'Prophet - Site aggregated','Seasonal Naive']

grid_level_fc_accuracy = pd.DataFrame(grid_results, index = column_names, columns = ['Grid Forecasts mean MASE'])

In [21]:
def highlight_min(s):
    is_min = grid_level_fc_accuracy.isin(grid_level_fc_accuracy.nsmallest(3, 'Grid Forecasts mean MASE')).values[:,0]
    return ['background-color: lightgreen' if v else '' for v in is_min]

In [22]:
grid_level_fc_accuracy.style.apply(highlight_min)

,Grid Forecasts mean MASE
LSTM - Grid,1.468234
LSTM - TL aggregated,1.416184
LSTM - SUB aggregated,1.330124
LSTM - PC aggregated,1.123227
LSTM - Site aggregated,1.013093
CNN - Grid,4.291811
CNN - TL aggregated,4.284695
CNN - SUB aggregated,4.287846
CNN - PC aggregated,1.228640
CNN - Site aggregated,1.252650


In [25]:
final_result_csv = grid_level_fc_accuracy.round(3)

In [26]:
final_result_csv.to_csv('final_results.csv')

In [27]:
# name = ['LSTM - Grid', 'LSTM - TL aggregated', 'LSTM - SUB aggregated', 'LSTM - PC aggregated', 
#                 'LSTM - Site aggregated', 'CNN - Grid', 'CNN - TL aggregated', 'CNN - SUB aggregated', 
#                 'CNN - PC aggregated', 'CNN - Site aggregated', 'Auto ARIMA - Grid']

fig = go.Figure()
count = 0
for col in grid_error_dist.keys():
    box_data = grid_error_dist[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean=True))
    count = count+1
    
for col in grid_error_dist_tcn.keys():
    box_data = grid_error_dist_tcn[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_arima.keys():
    box_data = grid_error_dist_arima[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_tbats.keys():
    box_data = grid_error_dist_tbats[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_prophet.keys():
    box_data = grid_error_dist_prophet[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    

box_data = grid_error_dist_naive['grid']
fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
count = count+1
    

fig.update_xaxes(title = '')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Grid level forecasts (MASE across 36 samples)", showlegend = False, font = dict(size=10))

In [31]:
x_vals = ['grid', 'tl', 'sub', 'postcode', 'site']
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=x_vals, y=grid_error),
    go.Bar(name='Dilated Convolutions', x=x_vals, y=grid_error_tcn)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='Grid level forecasts - Mean MASE (1 day ahead - 1 hour resolution)')
fig.show()

In [35]:
fig = go.Figure()
for col in grid_error_dist.keys():
    box_data = grid_error_dist[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'BU strategy - ts hierarchy level')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Grid level forecasts - Model LSTM", showlegend = False)

In [34]:
fig = go.Figure()
for col in grid_error_dist_tcn.keys():
    box_data = grid_error_dist_tcn[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'BU strategy - ts hierarchy level')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Grid level forecasts - Model Dilated Convolutions", showlegend=False)